In [1]:
import copy
import gym
import torch

import numpy as np
import torch.nn.functional as F

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import Tensor, nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from pytorch_lightning import LightningModule, Trainer

from gym.wrappers import RecordVideo, RecordEpisodeStatistics

In [2]:
def display_video(episode=0):
  video_file = open(f'/content/videos/rl-video-episode-{episode}.mp4', "r+b").read()
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"<video width=600 controls><source src='{video_url}'></video>")

In [3]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


- Create Deep Q Learning

In [4]:
class DQN (nn.Module):
    def __init__(self, n_actions, hidden_size, obs_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size), # Camada linear com 4 entradas e 8 saídas
            nn.ReLU(), # Função de ativação ReLU
            nn.Linear(hidden_size, hidden_size),# Camada linear com 4 entradas e 8 saídas
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions),
        )

    def forward(self,x):
        return self.net(x.float())

- Create a Policy

In [5]:
## Create a Policy
#  técnica amplamente usada em Reinforcement Learning (RL) para balancear exploração e exploração durante o treinamento de agentes.
def epsilon_greedy(state, env, net, epsilon=0.0):
    if np.random.random() < epsilon:
        action = env.action_space.sample()  # Escolhe uma ação aleatória
    else:
        state = torch.tensor([state]).to(device)  # Converte o estado para tensor e move para o dispositivo (CPU/GPU)
        q_values = net(state)  # Calcula os valores Q(s, a) usando a rede neural
        _, action = torch.max(q_values, dim=1)  # Seleciona a ação com o maior valor Q
        action = int(action.item())  # Converte a ação para um inteiro
    return action  # Retorna a ação escolhida

- Create Buffer replay

In [6]:
#Create Buffer replay
#O principal motivo do uso do Replay Buffer em algoritmos de Reinforcement Learning (RL), especialmente no Deep Q-Learning (DQN), 
#é quebrar a correlação entre as amostras de treinamento, 
#o que leva a um aprendizado mais estável e eficiente. 
#Ele também permite o reaproveitamento de experiências, reduzindo a necessidade de interagir continuamente com o ambiente.

class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)

    def __len__(self):
        return len(self.buffer)
    
    def append(self, experience):
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

In [7]:
class RLDataset(IterableDataset):
    def __init__(self, buffer, sample_size = 200): 
        self.buffer = buffer
        self.sample_size = sample_size
    

    def __iter__(self):
        for experience in self.buffer.sample(self.sample_size):
            yield experience

In [9]:
##Create Enviroment

def create_enviroment(name):
    env = gym.make(name)
    env = RecordVideo(env, video_folder='./videos')
    return env


env = create_enviroment("LunarLander-v2")


# Reset the environment to start
state = env.reset()
# Run for 1000 timesteps
for _ in range(1000):
    env.render()  # Render the environment
    action = env.action_space.sample()  # Take a random action
    print("Action taken:", action)


    # Do this action in the environment and get
    # next_state, reward, done and info
    _, observation, reward, done, info = env.step(action)
    print('Observation Space: ', observation)
    print('Reward: ', reward)


    # If the episode is done (CartPole has fallen), reset the environment
    if done:
        state = env.reset()

env.close()  # Close the rendering window



/home/igor/.local/lib/python3.10/site-packages/gym/wrappers/record_video.py:75: UserWarning: WARN: Overwriting existing videos at /home/igor/projects/reinforcement_learning/videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
/home/igor/.local/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:59: UserWarning: WARN: Disabling video recorder because environment <TimeLimit<OrderEnforcing<PassiveEnvChecker<LunarLander<LunarLander-v2>>>>> was not initialized with any compatible video mode between `rgb_array` and `rgb_array_list`
  logger.warn(
/home/igor/.local/lib/python3.10/site-packages/gym/envs/box2d/lunar_lander.py:604: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym("LunarLander-v2", render_mode="rgb_array")
  gym.logger.warn(


Action taken: 1
Observation Space:  -2.3257215714597224
Reward:  False
Action taken: 0
Observation Space:  -1.8902914429628765
Reward:  False
Action taken: 2
Observation Space:  0.46450346357813715
Reward:  False
Action taken: 1
Observation Space:  -2.965252440261905
Reward:  False
Action taken: 2
Observation Space:  0.27336144383160105
Reward:  False
Action taken: 3
Observation Space:  -1.378741428014307
Reward:  False
Action taken: 3
Observation Space:  -1.2609914788476988
Reward:  False
Action taken: 1
Observation Space:  -2.4740408439711232
Reward:  False
Action taken: 2
Observation Space:  0.11081373420470869
Reward:  False
Action taken: 0
Observation Space:  -1.891605607777052
Reward:  False
Action taken: 3
Observation Space:  -1.4408670848405325
Reward:  False
Action taken: 2
Observation Space:  1.0204856000315885
Reward:  False
Action taken: 2
Observation Space:  1.8275726348424086
Reward:  False
Action taken: 1
Observation Space:  -2.456804698028377
Reward:  False
Action taken